This notebook is motivated by the paper, [A Systematic Approach for Variable Selection
With Random Forests: Achieving Stable
Variable Importance Values](https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=8038868).  

In [10]:
# Get Table Data
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.names
# Get Table MetaData
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data
# Extract Feature Names 
!cat agaricus-lepiota.names | grep "^[[:space:]]\{4,5\}[0-9]\{1,2\}.*:" > agaricus-lepiota-feature-names.txt
# Move to _data folder (put _data in .gitignore file)
!mv agaricus* _data/

--2019-09-22 23:43:22--  https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.names
Resolving archive.ics.uci.edu... 128.195.10.252
Connecting to archive.ics.uci.edu|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6816 (6.7K) [application/x-httpd-php]
Saving to: 'agaricus-lepiota.names'

agaricus-lepiota.na 100%[===================>]   6.66K  --.-KB/s    in 0s      

2019-09-22 23:43:24 (43.0 MB/s) - 'agaricus-lepiota.names' saved [6816/6816]

--2019-09-22 23:43:24--  https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data
Resolving archive.ics.uci.edu... 128.195.10.252
Connecting to archive.ics.uci.edu|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 373704 (365K) [application/x-httpd-php]
Saving to: 'agaricus-lepiota.data'

agaricus-lepiota.da 100%[===================>] 364.95K   708KB/s    in 0.5s    

2019-09-22 23:43:25 (708 KB/s) - 'agar

I've worked with the UCI ML repository's mushroom data set [before](https://krbnite.github.io/Treebeard-and-the-Fungus-Amongus/), and I shamelessly borrow my code from there to import, transform, and split 
the data.

# Load Some Code

In [13]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import feature_selection as filt
%matplotlib inline

# Import Data

In [15]:
# Extract features names
with open('_data/agaricus-lepiota-feature-names.txt') as f:
    features = [line.split()[1][0:-1] for line in f]
    
# Get Data
data = pd.read_csv('_data/agaricus-lepiota.data', names=['deadly']+features)
y = pd.DataFrame([1 if target=='p' else 0 for target in data['deadly']], columns=['deadly'])
x = data.drop('deadly', axis=1)

# Split the Data

In [16]:
# Train, Validate, Test
x_trn, x_vt, y_trn, y_vt = train_test_split(x, y, train_size=0.70)
x_val, x_tst, y_val, y_tst = train_test_split(x_vt, y_vt, test_size=0.50)

# Some/Most techniques require the categorical vars to be one-hot encoded
x_trn_code = pd.get_dummies(x_trn)
x_val_code = pd.get_dummies(x_val)
x_tst_code = pd.get_dummies(x_tst)

# Make sure the various dummy vars are represented in each subset
all_ftrs = pd.get_dummies(x).columns
for ftr in all_ftrs.difference(x_trn_code.columns): x_trn_code[ftr]=0
for ftr in all_ftrs.difference(x_val_code.columns): x_val_code[ftr]=0
for ftr in all_ftrs.difference(x_tst_code.columns): x_tst_code[ftr]=0

/Users/kevinurban/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In the Behnamian paper, they do not toggle many hyperparameters:
* Leave mtry as default (sqrt(F))
* If you choose some kind of additional constraints, keep them fixed for all runs

Importantly, the authors looked at how many training iterations ("retrains" henceforth) it took 
for the average variable importance rankings to stabilize given RFs with 50, 200, 500, and 10k 
trees.  For each RF, they ran 25 retrains.

They looked at both Gini important (aka mean decrease in Gini impurity, MDG) and permutation
important (aka mean decrease in accuracy, MDA).

Basically, they look at sequences like
$$\{\overline{VI}_{p,nTree}(i)\}_{i=1}^{25}$$

where p is a fixed predictor, nTree is a fixed forest size, and the 
ith term is defined as

$$\overline{VI}_{p,nTree}(i) = \frac{1}{i}\sum\limits_{j=1}^{i}{VI_{p,nTree}(j)}$$

Plotting this sequence for a fixed predictor and forest size can shed light on how many
runs one should do to stabilize the variable importances.  Better, one might create a 
"spaghetti plot" of these sequences for, say, the top 10 predictors from the first training
iteration, then look to see when they all stop criss-crossing and seem to settle to a fixed
value.  

To have an aggregate indicator of stabilization over all predictors, the authors concocted
a distance metric:

$$D(i) = \sqrt{\frac{\sum_{p=1}^{P}(\overline{VI}_{p,nTree}(i) - \overline{VI}_{p,nTree}(25))^{2}}{P}}$$

This serves as an average "distance from the true mean" over all predictors, where the "true mean"
is assumed to be well approximated by the average over 25 retrains.

Their major finding was that a 10k-tree forest basically only needs one training iteration for
variable importance covergence, but that it is computationally burdensome compared to averaging, say,
the variable importances of a 50-tree forest over 21 retrains.  They quoted the timings for 
their data set, showing that the 10k-tree forest took 10x as long to train than the smaller 
forest over 21 retains.  

This is interesting because it really calls into question what you want to do
and which forest is best.  I've seen prediction accuracy stabilize at 50-100 trees,
while variable importances fluctuate like crazy from one training iteration to the
next w/ no change in hyperparameters.  This tells you that there are many ways to
skin a cat (that is, many ways to effectively separate classes), but that for a 
given predictor set you might imagine there is one "true way".  In the "true way" sense,
it "feels" like we should want to 10k-tree forest, but from a predictive accuracy, training
time, and prediction time sense, it is better to go with a smaller forest.  Maybe the best
of both worlds is finding the "true rankings", cutting the worst performers while maintaining
the same level of accuracy, then using as big a forest as convenient for the final forest.

For example, say you start with 100 predictors.  You find the true rankings using 30 retrains
of a fairly small forest.  Then you cut predictors from the bottom until the accuracy starts
to change.  Say after this, you have 35 predictors left.  At this point, opt for a slightly
bigger forest than you initially would have (e.g., with 100 predictors, you might have preferred
a 1k-tree forest, but could only afford a 500-tree forest in memory; well, go ahead and see
if you can afford that 1k-tree forest now, baby!).

